# Fonctions

In [2]:
import os
import re
from Bio import Entrez
import GEOparse
import gzip
import pandas as pd
import re
import json
import xml.etree.ElementTree as ET
from xml.etree import cElementTree as ElementTree
import matplotlib
Entrez.email = 'emailtest42aisd@outlook.com'

In [3]:
nb_retmax = 5000
handle = Entrez.esearch(db="gds", term="androgen receptor AND Homo Sapiens[porgn]", retmax = nb_retmax)
record = Entrez.read(handle)

In [4]:
def download_entry_GEO(GEO_id, directory): #télécharge Les résumés des entrées de La recherche par mots clés dans GEODataSets
    """"Crée un dossier contenant un dossier par identifiant et au sein de ce dossier
    crée le résumé (.txt) concernant l‘identifiant"""
    for entry in GEO_id:
        handle = Entrez.efetch(db="gds", id=[entry], retmode="text") #gds = banque de données GEO DataSets
        record = handle.read()
        record.encode("utf-8")
        path = directory+entry
        if not os.path.isdir(path):
            os.makedirs(path) #crée Le dossier Liés a L‘identifiant GEO
        fname = path+'/GEOentry'+entry+'.txt'
        with open(fname,'a', encoding="utf-8") as f : #rédaction du fichier texte résumant Les données de GEO datasets
            f.write(record)

In [5]:
def build_GEO_GSE(directory): #Lien entre Les identifiants GEO et Les numéros d'accés GSE
    """"Retourne le dictionnaire dict_GEO_GSE qui donne le lien entre GEO_id et GSEXXX.
    Prend en argument le chemin d'accés du dossier contenant les fichiers textes
    contenant les données GEO"""
    files = os.listdir(directory) #donne la Liste des fichiers
    dict_GEO_GSE = {}
    for entry in files:
        if entry != '.':
            fname = directory+'/'+entry+'/GEO'+entry+'.txt'
            # print (fname)
            resume = open(fname, "r", encoding="utf-8")
            all_lines = resume.readlines()
            resume.close()
            for line in all_lines:
                acc_nb = re.search('(GSE\d+)\s+', line) #recherche du numéros d'accés de La série GSE
                if acc_nb != None:
                    acc_nb = acc_nb.group(0)
                    acc = acc_nb.split("\t")
                    dict_GEO_GSE[entry] = acc[0].split()[0] #un unique numéro GSE par GEO_id
    return dict_GEO_GSE

In [6]:
def build_GSE_GEO(AR_GEO_GSE): #Lien entre Les numéros d'accés GSE et tes identifiants GEO
    """Retourne le dictionnaire dict_GSE_GEO en ayant pour argument le dictionnaire
    de lien entre GEO_id et GSEXXX (build_GEO_GSE())"""
    dict_GSE_GEO = {}
    for ID in AR_GEO_GSE:
        GSE_id = AR_GEO_GSE[ID]
        if GSE_id not in dict_GSE_GEO:
            dict_GSE_GEO[GSE_id] = [ID]
        else:
            dict_GSE_GEO[GSE_id].append(ID)
    return dict_GSE_GEO

In [7]:
def build_GSE_GEO(AR_GEO_GSE): #Lien entre Les numéros d'accés GSE et tes identifiants GEO
    """Retourne le dictionnaire dict_GSE_GEO en ayant pour argument le dictionnaire
    de lien entre GEO_id et GSEXXX (build_GEO_GSE())"""
    dict_GSE_GEO = {}
    for ID in AR_GEO_GSE:
        GSE_id = AR_GEO_GSE[ID]
        if GSE_id not in dict_GSE_GEO:
            dict_GSE_GEO[GSE_id] = [ID]
        else:
            dict_GSE_GEO[GSE_id].append(ID)
    return dict_GSE_GEO

In [8]:
def download_SOFT(AR_GSE_GEO, directory): #télécharge Les fichiers soft pour chacun des GSEXXX
    """Télécharge les fichiers .soft pour chacun des GSEXXX en ayant pour arguments le dictionnaire des
    liens entre GSE et GEO_id (build_GSE_GEO(AR_GEO_GSE)) et le chemin d'accés voulu
    gse_id_fails est une liste qui contient les entrées gse n‘ayant pas été téléchargées
    gse_id_success est une liste qui contient les entrées gse téléchargées"""
    gse_id_fails = []
    gse_id_success = []
    
    for gse_id in AR_GSE_GEO.keys():
        path = directory+gse_id
        gse_id = gse_id.strip()
        try:
        # gse_handle = GEOparse.get_GEO(geo = gse_id, destdir = path)
            print (" Downloading {0} \n".format(gse_id))
            gse_handle = GEOparse.get_GEO(geo = gse_id, destdir = path, silent = True)
            gse_id_success.append(gse_id)
        except :
            print (" !!! Download error with {0} \n".format(gse_id))
            gse_id_fails.append(gse_id)
    return (gse_id_success)

In [9]:
def read_file_SOFT(directory): #ouvre le fichier SOFT zippé et rend accessible ses données en chaines de caractéres
    """"Reatourne le fichier soft en liste de chaines de caractére ol chaque élément
    de la liste correspond & une ligne du fichier [‘lignei1', 'ligne2',...]. Prend
    en argument le chemin d'accés du fichier SOFT à analyser"""
    f = gzip.open(directory)
    file_content = f.read()
    file = str(file_content)
    all_lines = file.split('\\n')
    f.close()
    return all_lines

In [10]:
def info_sample(all_lines): #récupere Les informations Liees aux echantillons
    """"Retourne le dictionnaire contenant les informations des échantillons
    { "'GSMXxx" : [{titre : données},...], ‘GSMYYY' : [{titre : données},...]}.
    Prend en argument la lecture du fichier a analyser (fonction read_file_SOFT())"""

    dict_sample = {} #initialise Les dictionnaires qui vont contenir les infos {numéros d'accés : [infos]}
    l_sample = [] #initialise les Listes [infos] qui vont contenir Les dictionnaires {titre_info : données}
    entity_id = None #initialise L'identifiant de L'entité
    state = 'STATE_START' #initialise l'état du systéme
    kvp = re.compile(r'\!(.*) = (.*)')
    entity = re.compile(r'\^(.*) = (.*)')
    for line in all_lines: #on parcoure Les différentes Lignes du fichier
        if line.startswith('^'): #identifie La sous-structure associée a une entité (SERIES, PLATFORM ou SAMPLE)
            m = entity.match(line) #recherche L'expression réguliére
            if m:
                entity_type = m.group(1) #récupére Le titre de L’entité (SERIES, PLATFORM ou SAMPLE)
                entity_id = m.group(2) #récupere Le numéros d’accés associé @ L‘entité (GSEXXX, GPLXXX Ou GSMXXX)
                if entity_type == 'SAMPLE': #identifie le type SAMPLE
                    state = 'STATE_IN_SAMPLE'
                    sample_id = entity_id
                    if sample_id not in dict_sample: #crée une nouvelle clé pour un autre identifiant s'il y en a plusieurs
                        l_sample = []
                        dict_sample[sample_id] = l_sample
                        title = []
            else:
                state = 'STATE_START'
        elif line.startswith('!'): #identifie les Lignes correspondant aux données
            m = kvp.match(line) #recherche l’expression régulière
            if m:
                key = m.group(1) #récupére le titre de L'information de la ligne (ex : Serie_...)
                value = m.group(2) #récupére les données associées au titre
                if state == 'STATE_IN_SAMPLE': #lie les données Sample_... associées a L’entité SAMPLE et Les groupe dict_sample
                    if key not in title: #vérifie qu'il n'y a pas deux fois une clé avec Le méme nom
                        cpt = 1
                        title.append(key)
                        dict_sample[sample_id].append({key : value})
                    else:
                        cpt = cpt +1
                        topic = key + "_{0}".format(cpt) #renomme La clé si elle existe déja
                        dict_sample[sample_id].append({topic : value})
    return dict_sample

In [11]:
def info_serie_pm(all_lines):
    gse_pmid ={}
    entity = re.compile(r'\^SERIES = (.*)')
    pm_id = re.compile(r'\!Series_pubmed_id = (.*)')
#     for line in all_lines: #on parcoure Les différentes Lignes du fichier
#         if line.startswith('^'): #identifie La sous-structure associée a une entité (SERIES, PLATFORM ou SAMPLE)
#             gse_serie = entity.match(line) #recherche L'expression réguliére
#             if gse_serie:
#                 entity_type = gse_serie.group(1)
#             else:
#                 continue
    for line in all_lines:
        if pm_id.match(line):
            pubmed_id = pm_id.match(line)
            print(pubmed_id)
            pmid = pubmed_id.group(1)
            return pmid
        if not pm_id.match(line):
            pmid = ""
            continue
#             gse_pmid[entity_type]=pmid
#     gse_pmid[entity_type]=pmid
#   return gse_pmid

# Mise en pratique

In [12]:
download_entry_GEO(record['IdList'], 'scrapping/entry')

In [13]:
build_GSE_GEO(build_GEO_GSE('scrapping'))

{'GSE1561': ['entry1329', 'entry200001561'],
 'GSE3871': ['entry1836', 'entry200003871'],
 'GSE846': ['entry200000846', 'entry536'],
 'GSE4027': ['entry200004027', 'entry2034'],
 'GSE4084': ['entry200004084',
  'entry2384',
  'entry300092533',
  'entry300092534',
  'entry300092535',
  'entry300092536',
  'entry300092537',
  'entry300092538',
  'entry300092539',
  'entry300092540',
  'entry300092541',
  'entry300092542',
  'entry300092543',
  'entry300092544',
  'entry300092545',
  'entry300092546',
  'entry300092547',
  'entry300092548',
  'entry300092549',
  'entry300092550',
  'entry300092551',
  'entry300092552',
  'entry300092553',
  'entry300092554',
  'entry300092555',
  'entry300092556',
  'entry300092557',
  'entry300092558',
  'entry300092559',
  'entry300092560',
  'entry300092561',
  'entry300092562',
  'entry300092563',
  'entry300092564',
  'entry300092565',
  'entry300092566',
  'entry300092567',
  'entry300092568',
  'entry300092569',
  'entry300092570',
  'entry30009257

In [23]:
nb_gse = len(build_GSE_GEO(build_GEO_GSE('scrapping')))

In [14]:
nb_soft = download_SOFT(build_GSE_GEO(build_GEO_GSE('scrapping')),'./softFiles/soft')
nb_soft



 !!! Download error with GSE3871 





 !!! Download error with GSE4353 



 !!! Download error with GSE4453 




 !!! Download error with GSE4636 





D:\prog\Anaconda\envs\tdl\lib\site-packages\GEOparse\GEOparse.py:545: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  table_data = parse_table_data(gpl_soft)



 !!! Download error with GSE6796 


 !!! Download error with GSE6797 





 !!! Download error with GSE8402 



















 !!! Download error with GSE14464 




 !!! Download error with GSE17044 



 !!! Download error with GSE17466 



 !!! Download error with GSE18122 


 !!! Download error with GSE18146 




 !!! Download error with GSE18402 


 !!! Download error with GSE18684 


 !!! Download error with GSE19445 


 !!! Download error with GSE19561 


 !!! Download error with GSE21034 


 !!! Download error with GSE21035 



 !!! Download error with GSE21245 


 !!! Download error with GSE21887 





 !!! Download error with GSE22606 




 !!! Download error with GSE23815 




 !!! Download error with GSE27682 











 !!! Download error with GSE29232 


 !!! Download error with GSE29650 








 !!! Download error with GSE31978 



 !!! Download error with GSE32356 


 !!! Download error with GSE32875 









 !!! Download error with GSE35540 




 !!! Download erro

D:\prog\Anaconda\envs\tdl\lib\site-packages\GEOparse\GEOparse.py:545: DtypeWarning: Columns (11,14,15,36) have mixed types.Specify dtype option on import or set low_memory=False.
  table_data = parse_table_data(gpl_soft)





 !!! Download error with GSE47203 











 !!! Download error with GSE49083 



 !!! Download error with GSE49153 






 !!! Download error with GSE50618 







 !!! Download error with GSE51524 


 !!! Download error with GSE52169 



 !!! Download error with GSE52627 






 !!! Download error with GSE54137 


















 !!! Download error with GSE57820 



 !!! Download error with GSE58309 



 !!! Download error with GSE58474 



 !!! Download error with GSE58615 



 !!! Download error with GSE58812 





 !!! Download error with GSE60836 



 !!! Download error with GSE60842 










...

D:\prog\Anaconda\envs\tdl\lib\site-packages\GEOparse\GEOparse.py:545: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  table_data = parse_table_data(gpl_soft)


























 !!! Download error with GSE109763 















 !!! Download error with GSE115270 










 !!! Download error with GSE118261 


 !!! Download error with GSE118432 










 !!! Download error with GSE120006 

























































 !!! Download error with GSE138486 












































D:\prog\Anaconda\envs\tdl\lib\site-packages\GEOparse\GEOparse.py:545: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  table_data = parse_table_data(gpl_soft)


D:\prog\Anaconda\envs\tdl\lib\site-packages\GEOparse\GEOparse.py:545: DtypeWarning: Columns (2,4) have mixed types.Specify dtype option on import or set low_memory=False.
  table_data = parse_table_data(gpl_soft)


['GSE1561',
 'GSE846',
 'GSE4027',
 'GSE4084',
 'GSE4399',
 'GSE4454',
 'GSE4456',
 'GSE5345',
 'GSE5850',
 'GSE7585',
 'GSE7708',
 'GSE7868',
 'GSE8466',
 'GSE8533',
 'GSE8534',
 'GSE8860',
 'GSE9000',
 'GSE11155',
 'GSE11428',
 'GSE11847',
 'GSE12200',
 'GSE12348',
 'GSE12438',
 'GSE13332',
 'GSE13919',
 'GSE14028',
 'GSE14043',
 'GSE14092',
 'GSE14097',
 'GSE14575',
 'GSE15091',
 'GSE17461',
 'GSE18022',
 'GSE18338',
 'GSE18401',
 'GSE21036',
 'GSE22010',
 'GSE22076',
 'GSE22483',
 'GSE22914',
 'GSE23814',
 'GSE26483',
 'GSE27636',
 'GSE27823',
 'GSE28126',
 'GSE28219',
 'GSE28264',
 'GSE28596',
 'GSE28788',
 'GSE28857',
 'GSE28948',
 'GSE28950',
 'GSE30622',
 'GSE30623',
 'GSE30624',
 'GSE31362',
 'GSE31410',
 'GSE31528',
 'GSE32345',
 'GSE32892',
 'GSE32982',
 'GSE34042',
 'GSE34589',
 'GSE34780',
 'GSE35126',
 'GSE35211',
 'GSE35988',
 'GSE36526',
 'GSE37314',
 'GSE37345',
 'GSE38851',
 'GSE39357',
 'GSE39459',
 'GSE39645',
 'GSE39654',
 'GSE39879',
 'GSE40050',
 'GSE41784',
 'GS

*Fichiers soft dans un dossier/soft : soft+GSEID ; ex : softGSE14825*

*Fichiers softs : GSEID_family.soft contenus dans GSEID_family.soft.gz*

dict_full = {}

for GSEID in nb_soft:
    dict_full[GSEID] = info_sample(read_file_SOFT('softFiles/soft'+GSEID+'/'+GSEID+'_family.soft.gz'))

dict_full.keys()

In [15]:
dico_pmid = {}
for i in range(0, len(nb_soft)):
    dico_pmid[nb_soft[i]] = (info_serie_pm(read_file_SOFT('softFiles/soft'+nb_soft[i]+'/'+nb_soft[i]+'_family.soft.gz')))
print(dico_pmid)

<re.Match object; span=(0, 28), match='!Series_pubmed_id = 15897907'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 14702632'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 16707422'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 16707422'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 15767336'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 16932916'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 16932916'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 17263875'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 17148555'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 17426117'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 17566103'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 17679089'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 17510436'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 18923939'>
<re.Match object; sp

<re.Match object; span=(0, 28), match='!Series_pubmed_id = 24759320'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 25634993'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 24928520'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 24928520'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 25552417'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 26457646'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 25184630'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 25072326'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 25030374'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 25132238'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 25825716'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 29581250'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 29757368'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 29757368'>
<re.Match object; sp

<re.Match object; span=(0, 28), match='!Series_pubmed_id = 29899561'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 30085300'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 30016502'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 30016502'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 30016502'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 29977600'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 30827887'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 30827887'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 30827887'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 30183076'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 29808028'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 29808028'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 32350277'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 31266892'>
<re.Match object; sp

<re.Match object; span=(0, 28), match='!Series_pubmed_id = 34312180'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 33724955'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 34493786'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 33388344'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 33640491'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 34088870'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 34088870'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 34088870'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 33406413'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 33850637'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 34611310'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 33846123'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 33846123'>
<re.Match object; span=(0, 28), match='!Series_pubmed_id = 33846123'>
<re.Match object; sp

In [16]:
dico_pmid_clean = {k: v for k, v in dico_pmid.items() if v is not None}
dico_no_pmid = {k: v for k, v in dico_pmid.items() if v is None}

In [17]:
dico_pmid_clean

{'GSE1561': '15897907',
 'GSE846': '14702632',
 'GSE4027': '16707422',
 'GSE4084': '16707422',
 'GSE4399': '15767336',
 'GSE4454': '16932916',
 'GSE4456': '16932916',
 'GSE5345': '17263875',
 'GSE5850': '17148555',
 'GSE7585': '17426117',
 'GSE7708': '17566103',
 'GSE7868': '17679089',
 'GSE8466': '17510436',
 'GSE8860': '18923939',
 'GSE9000': '19176386',
 'GSE11155': '18632612',
 'GSE11428': '19632176',
 'GSE11847': '19330472',
 'GSE12200': '18852121',
 'GSE12348': '18974140',
 'GSE12438': '19176386',
 'GSE13332': '19118031',
 'GSE13919': '19244107',
 'GSE14028': '20478527',
 'GSE14043': '19318562',
 'GSE14092': '20478527',
 'GSE14097': '20478527',
 'GSE14575': '19345326',
 'GSE15091': '19668381',
 'GSE17461': '21592394',
 'GSE18338': '21307122',
 'GSE18401': '19832994',
 'GSE21036': '20579941',
 'GSE22010': '21747944',
 'GSE22076': '20610535',
 'GSE22483': '21330406',
 'GSE22914': '21829708',
 'GSE23814': '21592394',
 'GSE26483': '21212260',
 'GSE27636': '22373240',
 'GSE27823': '21

In [18]:
dico_no_pmid

{'GSE8533': None,
 'GSE8534': None,
 'GSE18022': None,
 'GSE31362': None,
 'GSE35211': None,
 'GSE43146': None,
 'GSE44264': None,
 'GSE44319': None,
 'GSE48667': None,
 'GSE51384': None,
 'GSE56188': None,
 'GSE60956': None,
 'GSE62442': None,
 'GSE64656': None,
 'GSE65562': None,
 'GSE66722': None,
 'GSE71334': None,
 'GSE71335': None,
 'GSE71336': None,
 'GSE72438': None,
 'GSE77770': None,
 'GSE77771': None,
 'GSE77883': None,
 'GSE86503': None,
 'GSE94520': None,
 'GSE98009': None,
 'GSE100224': None,
 'GSE103082': None,
 'GSE108687': None,
 'GSE110252': None,
 'GSE113308': None,
 'GSE114385': None,
 'GSE114500': None,
 'GSE114732': None,
 'GSE116189': None,
 'GSE116191': None,
 'GSE119338': None,
 'GSE123379': None,
 'GSE126318': None,
 'GSE127816': None,
 'GSE128018': None,
 'GSE131985': None,
 'GSE133119': None,
 'GSE138053': None,
 'GSE144326': None,
 'GSE148806': None,
 'GSE148807': None,
 'GSE148808': None,
 'GSE150475': None,
 'GSE152315': None,
 'GSE152318': None,
 'GSE156

In [27]:
nb_pubmed = len(set(dico_pmid_clean.values()))
nb_gse_publi = len(dico_pmid_clean)
nb_sans_publi = len(dico_no_pmid)
print(f'Avec {nb_gse} entrées GEO disponibles, ({len(nb_soft)} soft scrappés)')
print(f'Nous avons un total de {nb_gse_publi} set de données GEO avec publication Pubmed')
print(f'(Soit {nb_sans_publi} sans publications Pubmed)')
print(f'Pour un total de {nb_pubmed} publications Pubmed uniques')
print(f'Ayant pour identifiants pubmed (PMID) : {set(dico_pmid_clean.values())}')

Avec 530 entrées GEO disponibles, (469 soft scrappés)
Nous avons un total de 392 set de données GEO avec publication Pubmed
(Soit 77 sans publications Pubmed)
Pour un total de 274 publications Pubmed uniques
Ayant pour identifiants pubmed (PMID) : {'34439179', '23539448', '28609657', '31310591', '25908785', '27959342', '28891793', '30334814', '32531951', '33975627', '34493786', '33593085', '32199616', '34611310', '28775145', '23260764', '29921690', '28416760', '22722839', '15897907', '21330406', '26404510', '30395538', '30305041', '23817021', '33406413', '23717621', '31953400', '30930119', '33750801', '33038264', '29309643', '30100187', '30606742', '18632612', '33462444', '17566103', '32825120', '26219822', '23019221', '17263875', '19832994', '32198885', '33355184', '33687952', '33471819', '28893982', '31454442', '17679089', '28487115', '28428441', '19345326', '25482560', '21602269', '29142311', '22849360', '30853353', '21212260', '25065599', '24681825', '28059768', '21602788', '239027

# Pubmed

In [28]:
if not os.path.isdir('pubmed'):
    os.mkdir("pubmed")
    
for gse in dico_pmid_clean :
    pmid = dico_pmid_clean.get(gse)
    handle = Entrez.efetch(db = 'pubmed', id = pmid, rettype = 'full', retmode = 'xml')
    
    fname = "pubmed/"+gse+"_pmid_"+dico_pmid_clean.get(gse)+"_article.xml"
    
    tree = ET.parse(handle)
    tree.write(fname)

In [30]:
pubmed_dates = {}
liste_date = []
liste_mois = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

for gse in dico_pmid_clean:
    pmid = dico_pmid_clean.get(gse)
    tree = ElementTree.parse("pubmed/"+gse+"_pmid_"+dico_pmid_clean.get(gse)+"_article.xml")
    root = tree.getroot()
    
    #Dictionnaire Dates de publications
    publication = root.findall(".//PubDate")
    for dates in publication :
        year = int(dates.find('Year').text)
        if dates.find('Month') is not None :
            month = dates.find('Month').text
            for mois in liste_mois :
                if month == mois :
                    month = liste_mois.index(mois)
            month = int(month)
        else :
            month = 0
            
        pubmed_dates[pmid]=year,month
        
    
    #Dictionnaire Grant :
    
    #Dictionnaire méthodes :

print(pubmed_dates)
type(pubmed_dates['15897907'])

{'15897907': (2005, 6), '14702632': (2004, 0), '16707422': (2006, 4), '15767336': (2005, 2), '16932916': (2006, 10), '17263875': (2007, 0), '17148555': (2007, 1), '17426117': (2007, 0), '17566103': (2007, 5), '17679089': (2007, 7), '17510436': (2007, 4), '18923939': (2007, 0), '19176386': (2009, 1), '18632612': (2008, 6), '19632176': (2009, 6), '19330472': (2009, 5), '18852121': (2008, 9), '18974140': (2008, 10), '19118031': (2009, 0), '19244107': (2009, 2), '20478527': (2010, 4), '19318562': (2009, 3), '19345326': (2009, 3), '19668381': (2009, 7), '21592394': (2011, 4), '21307122': (2011, 4), '19832994': (2009, 9), '20579941': (2010, 6), '21747944': (2011, 0), '20610535': (2010, 7), '21330406': (2011, 3), '21829708': (2011, 0), '21212260': (2011, 2), '22373240': (2012, 1), '21572438': (2011, 4), '21602788': (2011, 4), '23260764': (2013, 0), '22083957': (2012, 0), '21741601': (2011, 6), '21602269': (2011, 6), '22531786': (2012, 5), '21915096': (2011, 8), '22014572': (2011, 9), '2518935

tuple

# Visu